<a href="https://colab.research.google.com/github/lihua1018/C--CourseSystemApp/blob/main/Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 台股清單

In [ ]:
!pip install pandas
!pip install requests
!pip install pathlib

In [ ]:
from pathlib import Path

# Create data folder if not exist
Path("data").mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
import requests

link = "https://quality.data.gov.tw/dq_download_json.php?nid=11549&md5_url=bb878d47ffbe7b83bfc1b41d0b24946e"

# Slice to Stock list
df = pd.DataFrame(requests.get(link).json())[["證券代號", "證券名稱"]]
df = df.rename(columns={"證券代號": "STOCK_ID", "證券名稱": "STOCK_NAME"})

df.to_csv("data/stock_id.csv", index=False, header=True)
df.head()

,證券代號,證券名稱,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,0050,元大台灣50,6315479,674047631,106.35,107.05,106.30,106.70,0.6500,7927
1,0051,元大中型100,25690,1026162,39.85,40.00,39.85,39.90,0.2500,30
2,0052,富邦科技,1126837,105810332,94.20,94.30,93.50,93.70,0.2000,250
3,0053,元大電子,10080,515800,50.95,51.30,50.95,51.30,0.6000,11
4,0054,元大台商50,24000,611660,25.45,25.69,25.45,25.69,0.3900,5
...,...,...,...,...,...,...,...,...,...,...
1118,9944,新麗,168000,3738700,22.05,22.45,22.05,22.35,0.0500,100
1119,9945,潤泰新,3801519,153073013,40.40,40.55,40.05,40.05,-0.2000,1863
1120,9946,三發地產,104096,1929791,18.35,19.00,18.35,18.55,0.0000,43
1121,9955,佳龍,191342,3619463,18.90,19.00,18.80,18.85,0.1000,104


# ETF50

In [ ]:
!pip install urllib3
!pip install bs4

ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [ ]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from bs4 import BeautifulSoup
import unicodedata as ucd

In [ ]:
# Request html
retry_strategy = Retry(total=3)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)
response = http.get("https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW")

# Parser html
soup = BeautifulSoup(response.content, "html.parser")

df = pd.DataFrame()
row_index = 0

# Locate the table by find the sibling html tag which have id attribute
first_table = soup.find(id="ctl00_ctl00_MainContent_MainContent_sdate3").find_next_sibling()
stock_tag = first_table.find_all("td")
for i in range(0, len(stock_tag), 4):
    stock_name = stock_tag[i].text.strip()

    df.loc[row_index, "STOCK_NAME"] = stock_name
    df.loc[row_index, "持股(千股)"] = stock_tag[i + 1].text.strip()
    df.loc[row_index, "比例"] = stock_tag[i + 2].text.strip()
    df.loc[row_index, "增減"] = stock_tag[i + 3].text.strip()
    row_index += 1

stock_tag = first_table.find_next_sibling().find_all("td")

for i in range(0, len(stock_tag), 4):
    stock_name = stock_tag[i].text.strip()

    df.loc[row_index, "STOCK_NAME"] = stock_name
    df.loc[row_index, "持股(千股)"] = stock_tag[i + 1].text.strip()
    df.loc[row_index, "比例"] = stock_tag[i + 2].text.strip()
    df.loc[row_index, "增減"] = stock_tag[i + 3].text.strip()
    row_index += 1


# Combine with Stock ID
stock_df = pd.read_csv("data/stock_id.csv")
result_df = pd.merge(df, stock_df, how="left", on=["STOCK_NAME"])
result_df = result_df[["STOCK_ID", "STOCK_NAME", "持股(千股)", "比例", "增減"]]

In [ ]:
result_df.head()

,STOCK_ID,STOCK_NAME,持股(千股),比例,增減
0,2330,台積電,"147,631.00",46.93,-0.18%
1,2454,聯發科,"9,103.00",4.54,-0.15%
2,2317,鴻海,"73,314.00",4.42,+0.01%
3,2303,聯電,"71,656.00",2.27,+0.23%
4,2308,台達電,"13,287.00",2.09,-0.07%


In [ ]:
result_df.to_csv("data/ETF50.csv", index=False, header=True)

# 台股歷史交易資料

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
# 護國神股
TSMC = yf.Ticker("2330.TW")

# 股利
TSMC.dividends

# Dividends & StockSplits
TSMC.actions

#%% 報表
TSMC.balance_sheet.index

#%% 歷史紀錄
TSMC.calendar

#%% 金流
TSMC.cashflow

#%% 收益
TSMC.earnings

#%% 財報
TSMC.financials


In [ ]:
etf50_df = pd.read_csv("data/ETF50.csv")
etf50_id = etf50_df.loc[:, "STOCK_ID"].astype(str) + ".TW"
etf50_id = etf50_id.str.cat(sep=" ")
print(etf50_id)

2330.TW 2454.TW 2317.TW 2303.TW 2308.TW 2881.TW 1303.TW 1301.TW 2002.TW 2882.TW 2412.TW 3711.TW 2891.TW 2886.TW 1216.TW 2603.TW 2884.TW 3008.TW 2885.TW 3034.TW 5871.TW 1326.TW 2379.TW 1101.TW 2327.TW 2615.TW 2357.TW 2892.TW 6415.TW 5880.TW 2382.TW 2207.TW 2880.TW 2409.TW 2609.TW 2887.TW 3045.TW 2912.TW 2395.TW 5876.TW 4938.TW 6505.TW 1590.TW 1102.TW 1402.TW 2801.TW 9910.TW 4904.TW 8046.TW 2408.TW


In [ ]:
# Download etf50 recent 1 years data
df = yf.download(etf50_id, group_by="Ticker", period="1y", interval="1d")

# rotate Ticker axis and convert to (Date,Ticker) index
df = df.stack(level=0).rename_axis(["Date", "Ticker"]).reset_index(level=1)

# drop index
df = df.reset_index(level=0)
df = df.rename(columns={"Ticker": "STOCK_ID"})
df = df[["STOCK_ID", "Date", "Adj Close", "Close", "High", "Low", "Open", "Volume"]]

[*********************100%***********************]  50 of 50 completed


In [ ]:
df.head()

,STOCK_ID,Date,Adj Close,Close,High,Low,Open,Volume
0,1101.TW,2011-09-08,17.105333,29.652229,29.975855,28.721806,28.964523,34062778.0
1,1102.TW,2011-09-08,20.104084,33.407612,33.920860,32.941029,33.174320,15081867.0
2,1216.TW,2011-09-08,24.594574,33.150928,33.270893,32.591080,33.270893,7903412.0
3,1301.TW,2011-09-08,54.985943,81.346153,83.557693,80.865387,82.884613,17639440.0
4,1303.TW,2011-09-08,44.049244,66.039604,68.118813,66.039604,67.524750,12772460.0


In [ ]:
df.to_csv("data/ETF50_10years.csv", index=False, header=True)

# 三大法人(RIP)

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
from datetime import date, timedelta
from pathlib import Path
from tqdm import tqdm
import requests
from io import StringIO

Path("data/TII").mkdir(parents=True, exist_ok=True)

In [ ]:
hack_date = date.today()
for i in tqdm(range(365)):
    hack_date_str = hack_date.strftime("%Y%m%d")

    # print(f"Current Date: {hack_date_str}")

    url = f"https://www.twse.com.tw/fund/T86?response=csv&date={hack_date_str}&selectType=ALL"

    if len(requests.get(url).text) > 2:
        data = StringIO(requests.get(url).text)
        df = pd.read_table(data, sep=",", skiprows=[0])
        hack_date -= timedelta(days=1)
    else:
        hack_date -= timedelta(days=1)
        continue

    df = df.drop(df.tail(8).index).drop(columns=df.columns[-1], axis=1)
    df.to_csv(f"data/TII/{hack_date_str}.csv", index=False, header=True)
    # time.sleep(60)


  0%|          | 0/365 [00:00<?, ?it/s]

Current Date: 20210909


  0%|          | 1/365 [00:28<2:53:43, 28.64s/it]

Current Date: 20210908


  1%|          | 2/365 [00:56<2:50:59, 28.26s/it]

...

 99%|█████████▉| 363/365 [1:17:24<00:15,  7.62s/it]

Current Date: 20200911


100%|█████████▉| 364/365 [1:17:39<00:09,  9.89s/it]

Current Date: 20200910


100%|██████████| 365/365 [1:17:54<00:00, 12.81s/it]
